In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [49]:
# import tensorflow as tf
# from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [50]:
# Set random seed for reproducibility
tf.random.set_seed(42)

# Define parameters
IMG_SIZE = (180,180)
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 5
_input = Input((180,180,3))

In [51]:
# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'Training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'Training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)


Found 4100 images belonging to 5 classes.
Found 1022 images belonging to 5 classes.


In [52]:
# Test data generator (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'Testing_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)


Found 1275 images belonging to 5 classes.


In [53]:
from tensorflow.keras.models import Sequential

In [43]:
# model_classify = Sequential([
#     layers.Rescaling(1./255),
#     layers.Conv2D(16,3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32,3,padding='same',activation='relu'),
#     layers.MaxPool2D(),
#     layers.Conv2D(64,3,padding='same',activation='relu'),
#     layers.MaxPool2D(),
#     layers.Conv2D(32,3,padding='same',activation='relu'),
#     layers.MaxPool2D(),
#     layers.Flatten(),
#     layers.Dropout(0.2),
#     layers.Dense(128),
#     layers.Dense(units=NUM_CLASSES)
# ])

conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(4096, activation="relu")(flat)
dense2 = Dense(4096, activation="relu")(dense1)
output = Dense(5, activation="softmax")(dense2)

vgg16_model  = Model(inputs=_input, outputs=output)

# model = Sequential([
#     layers.Rescaling(1./255),
#     layers.Conv2D(32, 3, padding='same', activation='relu'),
#     layers.BatchNormalization(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(),
#     layers.Conv2D(128, 3, padding='same', activation='relu'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(),
#     layers.Conv2D(256, 3, padding='same', activation='relu'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dropout(0.2),
#     layers.Dense(256, activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(0.2),
#     layers.Dense(units=len(data_cat), activation='softmax')
# ])

In [44]:
vgg16_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 conv2d_34 (Conv2D)          (None, 180, 180, 64)      1792      
                                                                 
 conv2d_35 (Conv2D)          (None, 180, 180, 64)      36928     
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 90, 90, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 90, 90, 128)       73856     
                                                                 
 conv2d_37 (Conv2D)          (None, 90, 90, 128)       147584    
                                                             

In [45]:
from tensorflow.keras.optimizers import Adam


vgg16_model.compile(optimizer=Adam(), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [47]:
# epochs=25
history =vgg16_model.fit(history =model_classify.fit(train_generator,validation_data=validation_generator,epochs=20,batch_size=32))

Epoch 1/20
129/129 [==============================] - 40s 311ms/step - loss: 1.3569 - accuracy: 0.5220 - val_loss: 1.3565 - val_accuracy: 0.5225
Epoch 2/20
129/129 [==============================] - 39s 304ms/step - loss: 1.3570 - accuracy: 0.5220 - val_loss: 1.3556 - val_accuracy: 0.5225
Epoch 3/20
129/129 [==============================] - 40s 310ms/step - loss: 1.3566 - accuracy: 0.5220 - val_loss: 1.3571 - val_accuracy: 0.5225
Epoch 4/20
129/129 [==============================] - 45s 345ms/step - loss: 1.3572 - accuracy: 0.5220 - val_loss: 1.3547 - val_accuracy: 0.5225
Epoch 5/20
129/129 [==============================] - 88s 687ms/step - loss: 1.3569 - accuracy: 0.5220 - val_loss: 1.3550 - val_accuracy: 0.5225
Epoch 6/20
129/129 [==============================] - 53s 413ms/step - loss: 1.3576 - accuracy: 0.5220 - val_loss: 1.3558 - val_accuracy: 0.5225
Epoch 7/20
129/129 [==============================] - 50s 391ms/step - loss: 1.3567 - accuracy: 0.5220 - val_loss: 1.3558 - val_ac

KeyboardInterrupt: 

In [25]:
from tensorflow.keras.optimizers import Adam


model_classify.compile(optimizer=Adam(), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [26]:
history =model_classify.fit(train_generator,validation_data=validation_generator,epochs=20,batch_size=32
                  )

Epoch 1/20
129/129 [==============================] - 103s 782ms/step - loss: 1.3765 - accuracy: 0.5185 - val_loss: 1.3600 - val_accuracy: 0.5225
Epoch 2/20
129/129 [==============================] - 43s 329ms/step - loss: 1.3627 - accuracy: 0.5220 - val_loss: 1.3563 - val_accuracy: 0.5225
Epoch 3/20
129/129 [==============================] - 53s 409ms/step - loss: 1.3642 - accuracy: 0.5220 - val_loss: 1.3546 - val_accuracy: 0.5225
Epoch 4/20
129/129 [==============================] - 38s 295ms/step - loss: 1.3586 - accuracy: 0.5220 - val_loss: 1.3579 - val_accuracy: 0.5225
Epoch 5/20
129/129 [==============================] - 44s 344ms/step - loss: 1.3590 - accuracy: 0.5220 - val_loss: 1.3578 - val_accuracy: 0.5225
Epoch 6/20
129/129 [==============================] - 40s 307ms/step - loss: 1.3621 - accuracy: 0.5220 - val_loss: 1.3565 - val_accuracy: 0.5225
Epoch 7/20
129/129 [==============================] - 42s 323ms/step - loss: 1.3572 - accuracy: 0.5220 - val_loss: 1.3577 - val_a

KeyboardInterrupt: 

In [61]:
model =Sequential()

model.add(Conv2D(32,kernel_size=(3,3), padding ='valid',activation='relu', input_shape=(180,180,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3), padding ='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3), padding ='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='sigmoid'))

In [62]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_50 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 89, 89, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_51 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 43, 43, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_52 (Conv2D)          (None, 41, 41, 128)       73856     
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 20, 20, 128)     

In [59]:
from tensorflow.keras.optimizers import Adam


model.compile(optimizer=Adam(), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history =model.fit(train_generator,validation_data=validation_generator,epochs=20,batch_size=32
                  )